# Построение НКА по регулярному выражению

In [128]:
def star(graph: dict,
         symbol: str,
         cur_state: int):
    graph[f"A{cur_state}"] = [("EMPTY", f"A{cur_state + 1}")]
    cur_state += 1

    if len(symbol) == 1:
        graph[f"A{cur_state}"] = [(symbol, f"A{cur_state}")]
        cur_state += 1
    elif len(symbol) > 1:
        start_state = cur_state
        for i in symbol:
            graph[f"A{cur_state}"] = [(i, f"A{cur_state + 1}")]
            cur_state += 1
        
        graph[f"A{cur_state - 1}"].append((symbol[0], f"A{start_state}"))
    
    graph[f"A{cur_state - 1}"].append(("EMPTY", f"A{cur_state}"))

    return graph, cur_state


def question(graph: dict,
             symbol: str,
             cur_state: int):
    if len(symbol) == 1:
        graph[f"A{cur_state}"] = [(symbol, f"A{cur_state + 1}")]
        cur_state += 1

        graph[f"A{cur_state - 1}"].append(("EMPTY", f"A{cur_state}"))
    elif len(symbol) > 1:
        start_state = cur_state

        for i in symbol:
            graph[f"A{cur_state}"] = [(i, f"A{cur_state + 1}")]
            cur_state += 1
        
        graph[f"A{start_state}"].append(("EMPTY", f"A{cur_state}"))

    return graph, cur_state


def plus(graph: dict,
         symbol: str,
         cur_state: int):
    if len(symbol) == 1:
        graph[f"A{cur_state}"] = [(symbol, f"A{cur_state + 1}")]
        cur_state += 1
        graph[f"A{cur_state}"] = [(symbol, f"A{cur_state}")]
        cur_state += 1
    elif len(symbol) > 1:
        start_state = cur_state
        for i in symbol:
            graph[f"A{cur_state}"] = [(i, f"A{cur_state + 1}")]
            cur_state += 1
        
        graph[f"A{cur_state - 1}"].append((symbol[0], f"A{start_state}"))
    
    graph[f"A{cur_state - 1}"].append(("EMPTY", f"A{cur_state}"))

    return graph, cur_state


def slash(graph: dict,
          symbol: str,
          cur_state: int):
    pass

In [129]:
def create_graph(s: str):
    cur_state = 0
    graph = dict()

    for i in range(len(s)):
        if i + 1 < len(s) and s[i + 1] == '*':
            graph, cur_state = star(graph, s[i], cur_state)
        elif i + 1 < len(s) and s[i + 1] == '+':
            graph, cur_state = plus(graph, s[i], cur_state)
        elif i + 1 < len(s) and s[i + 1] == '?':
            graph, cur_state = question(graph, s[i], cur_state)
        elif i + 2 < len(s) and s[i + 1] == '|':
            graph, cur_state = slash(graph, s[i], s[i + 2], cur_state)
        elif s[i] in ('*', '+', '?', '|'):
            continue
        else:
            graph[f"A{cur_state}"] = [(s[i], f"A{cur_state + 1}")]
            cur_state += 1
    
    return graph, f"A{cur_state}"

In [130]:
s = "he+n?l*o"
nka, fin_state = create_graph(s)
nka

{'A0': [('h', 'A1')],
 'A1': [('e', 'A2')],
 'A2': [('e', 'A2'), ('EMPTY', 'A3')],
 'A3': [('n', 'A4'), ('EMPTY', 'A4')],
 'A4': [('EMPTY', 'A5')],
 'A5': [('l', 'A5'), ('EMPTY', 'A6')],
 'A6': [('o', 'A7')]}

In [131]:
fin_state

'A7'

# Избавление от е-дуг

In [133]:
def check_depth(graph: dict,
                cur_state: str,
                depth: int = 0,
                states: list = list()
                ) -> list:
    if cur_state in graph.keys():
        if depth == 0:
            for transit in graph[cur_state]:
                if transit[1] not in states and transit[0] != "EMPTY":
                    states.append(transit[1])
                    states.extend(check_depth(graph, transit[1], depth + 1, states))
        else:
            for transit in graph[cur_state]:
                if transit[1] not in states and transit[0] == "EMPTY":
                    if transit[1] not in states:
                        states.append(transit[1])
                        states.extend(check_depth(graph, transit[1], depth + 1, states))

    states = list(set(states))
    
    return states

In [137]:
check_depth(nka, "A1", depth=0, states=list())

['A3', 'A2', 'A5', 'A6', 'A4']

In [144]:
def delete_missing_states(graph: dict) -> dict:
    res = dict()

    missing = list()
    for state in graph.keys():
        if len(graph[state]) == 0:
            missing.append(state)
    
    for state in missing:
        graph.pop(state)

    for state in graph.keys():
        res[state] = list()
        for transit in graph[state]:
            if transit[1] not in missing:
                res[state].append(transit)
    
    return res

In [157]:
def lose_emptys(graph: dict,
                fin_state: str):
    states = dict()
    emptys = dict()
    for state in graph.keys():
        states[state] = list()
        emptys[state] = check_depth(graph, state, depth=0, states=list())

    for state in states.keys():
        for transit in graph[state]:
            if transit[0] != "EMPTY":
                for emp in emptys[state]:
                    states[state].append((transit[0], emp))
    
    states = delete_missing_states(states)

    return states

In [158]:
lose_emptys(nka, fin_state)

{'A0': [('h', 'A1')],
 'A1': [('e', 'A3'), ('e', 'A2'), ('e', 'A5'), ('e', 'A6')],
 'A2': [('e', 'A3'), ('e', 'A2'), ('e', 'A5'), ('e', 'A6')],
 'A3': [('n', 'A5'), ('n', 'A6')],
 'A5': [('l', 'A5'), ('l', 'A6')],
 'A6': [('o', 'A7')]}

# Преобразование НКА в ДКА